In [1]:
!pip install langchain chromadb openai tiktoken
!pip install -U langchain-community
!pip install -U unstructured
!pip install -U langchain-huggingface


In [2]:
speech="/kaggle/input/speech/speech.txt"

from langchain_community.document_loaders import TextLoader
loader=TextLoader(speech)
text_documents=loader.load()
text_documents


[Document(metadata={'source': '/kaggle/input/speech/speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of 

In [2]:
!pip install pypdf
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader("/kaggle/input/attention/attention.pdf")
doc=loader.load()

In [3]:
#dividing into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
document=text_splitter.split_documents(doc)
document[:5]

[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-08-03T00:07:29+00:00', 'author': '', 'keywords': '', 'moddate': '2023-08-03T00:07:29+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': '/kaggle/input/attention/attention.pdf', 'total_pages': 15, 'page': 0, 'page_label': '1'}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.edu\nŁukasz Kaiser

# Using Embeddings

# **all-MiniLM-L6-v2 Embeddings**

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.schema import Document

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
docs = [Document(page_content=doc.page_content, metadata={"index": i}) for i, doc in enumerate(document[:20])]
db = Chroma.from_documents(docs, embedding_model)

print("ChromaDB created with updated HuggingFace embeddings!")


In [ ]:
query="who are the authors of attention is all you need research paper?"
result=db.similarity_search(query)
result[0].page_content
#not accurate Result

# Delete the already made chromaDB

In [11]:
import shutil
# Delete the old ChromaDB folder
shutil.rmtree("./chroma_db", ignore_errors=True)
# reloading ChromaDB after deletion
db = None  
print("ChromaDB reference cleared.")

ChromaDB reference cleared.


In [12]:
if db is not None:
    print("Number of documents in ChromaDB:", db._collection.count())
    embeddings_data = db._collection.get(include=["embeddings"])
    
    if "embeddings" in embeddings_data and embeddings_data["embeddings"]:
        sample_embedding = embeddings_data["embeddings"][0]
        print("Embedding Dimension:", len(sample_embedding))
    else:
        print("No embeddings found in ChromaDB.")
else:
    print("No ChromaDB reference found.")

No ChromaDB reference found.


In [10]:
from chromadb import PersistentClient
# Connect to your ChromaDB
client = PersistentClient(path="./chroma_db")

# List existing collections
print("Available Collections:", client.list_collections())


Available Collections: []


In [ ]:
# Not Working
# Delete a specific collection
collection_name = "langchain"  
client.delete_collection(name=collection_name)

print(f" Collection '{collection_name}' deleted successfully!")


# **intfloat/e5-large-v2 Embeddings**

In [6]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.schema import Document

embedding_model = HuggingFaceEmbeddings(model_name="intfloat/e5-large-v2")
docs = [Document(page_content=doc.page_content, metadata={"index": i}) for i, doc in enumerate(document[:20])]
db = Chroma.from_documents(docs, embedding_model)

print(" ChromaDB created with updated HuggingFace embeddings!")

 ChromaDB created with updated HuggingFace embeddings!


In [7]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.schema import Document

embedding_model = HuggingFaceEmbeddings(model_name="intfloat/e5-large-v2")
docs = [Document(page_content=doc.page_content, metadata={"index": i}) for i, doc in enumerate(document[:20])]
db = Chroma.from_documents(docs, embedding_model)

print(" ChromaDB created with updated HuggingFace embeddings!")
def search_query():
    query = input("Enter your search query: ")
    
    result = db.similarity_search(query, k=3)  

    if result:
        print("\n **Top 3 Matching Results:**\n")
        for i, doc in enumerate(result, start=1):
            print(f"{i}. {doc.page_content}\n" + "-"*50)
    else:
        print("\n No relevant result found.")

#who are the authors of attention is all you need?

In [8]:
search_query()

Enter your search query:  who are the authors of attention is all you need?



 **Top 3 Matching Results:**

1. Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.com
Noam Shazeer∗
Google Brain
noam@google.com
Niki Parmar∗
Google Research
nikip@google.com
Jakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.com
Aidan N. Gomez∗ †
University of Toronto
aidan@cs.toronto.edu
Łukasz Kaiser∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗ ‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurre

# Using GloVe Embeddings

In [ ]:
#use only if you want to dowmload, you can download once and then upload it as dataset
#!wget https://nlp.stanford.edu/data/glove.6B.zip

In [ ]:
'''!unzip glove.6B.zip
!ls'''

In [15]:
import numpy as np
# Load GloVe embeddings
def load_glove_embeddings(file_path):
    embeddings_index = {}
    with open(file_path, "r", encoding="utf8") as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.array(values[1:], dtype="float32")
            embeddings_index[word] = vector
    return embeddings_index

# Load 300-dimensional GloVe embeddings
glove_path = "/kaggle/input/glove-embeddings/glove.6B.300d.txt"
glove_embeddings = load_glove_embeddings(glove_path)

print(f" Loaded {len(glove_embeddings)} word vectors!")


 Loaded 400000 word vectors!


In [16]:
# Function to compute document embedding (average word vectors)
def get_document_embedding(text, glove_embeddings, dim=300):
    words = text.lower().split()
    word_vectors = [glove_embeddings[word] for word in words if word in glove_embeddings]

    if len(word_vectors) == 0:
        return np.zeros(dim)  # Return zero vector if no words found

    return np.mean(word_vectors, axis=0)  # Average word vectors


In [17]:
from langchain.embeddings.base import Embeddings

# Custom embedding function for ChromaDB
class GloVeEmbeddings(Embeddings):
    def embed_documents(self, texts):
        return [get_document_embedding(text, glove_embeddings).tolist() for text in texts]

    def embed_query(self, text):
        return get_document_embedding(text, glove_embeddings).tolist()


In [18]:
from langchain_community.vectorstores import Chroma
from langchain.schema import Document

# Convert raw text documents into LangChain Document objects
docs = [Document(page_content=doc.page_content, metadata={"index": i}) for i, doc in enumerate(document[:20])]

# Initialize ChromaDB with GloVe embeddings
embedding_model = GloVeEmbeddings()
db = Chroma.from_documents(docs, embedding_model, persist_directory="./chroma_db")

print(" ChromaDB created with GloVe embeddings!")


OperationalError: attempt to write a readonly database

In [ ]:
search_query()
